In [55]:
max1demand=10    #use 1 more than actual demand
max2demand=10    #use 1 more than actual demand
sparti=[]  #start with null #ZoneActiveStateDefinition 1-Not active in 1; 2-Not active in 2; 3- Active in 1; 4- Active in 2;
spartj=[]  #start with null #Zone1DemandStateDef
spartk=[]  #start with null #Zone2DemandStateDef
spartl=[]  #start with null #PreviousZoneStateDef
sidx=[]    #start with null
s=1        #start number for loop
for i in 1:4           #Create an index to call   
    for j in 1:max1demand
        for k in 1:max2demand
            for l in 1:2 #2 regions
                sparti= push!(sparti,i)  #for i in ijk add to sparti definition
                spartj= push!(spartj,j)  #for j in ijk add to spartj definition
                spartk= push!(spartk,k)  #for k in ijk add to spartk definition
                spartl= push!(spartl,l)  #for k in ijk add to spartk definition
                sidx= push!(sidx,s)      #create index in sidx
                s=s+1
            end
        end
    end
end

#sparti
#spartj
#spartk
#[sparti,spartj,spartk]
#sidx

#idxactive=  (sparti.>=3) .& (spartj+spartk.==2) ##(Wrapped into regional indices)
idxidle= (sparti.<3) .& (spartj+spartk .>2)   #index all ijk where idling cannot occur
idxZoneFrom= (sparti.==1) .& (spartl.==2) .| (sparti.==2) .& (spartl.==1) #idx where zone from cannot occur
idxregion1= (sparti.==3) .& (spartj.==1)      #index all ijk where region 1 is active but there are no calls in region 1 to service
idxregion2= (sparti.==4) .& (spartk.==1)      #index all ijk where region 2 is active but there are no calls in region 2 to service


idxDEL=idxidle .| idxregion1 .| idxregion2 .| idxZoneFrom   #index of the previous 3 indices

deleteat!(sparti, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartj, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartk, BitArray(idxDEL)) #delete idxDEL
deleteat!(spartl, BitArray(idxDEL)) #delete idxDEL
sidx= [1:length(sparti);];          #number of states
sidx

362-element Vector{Int64}:
   1
   2
   3
   4
   5
   6
   7
   8
   9
  10
  11
  12
  13
   ⋮
 351
 352
 353
 354
 355
 356
 357
 358
 359
 360
 361
 362

In [78]:
λ=1.75
μS=1.75
μD=1
μMatr=[5 4; 
        4 5]

2×2 Matrix{Int64}:
 5  4
 4  5

In [79]:
matrix= zeros(length(sidx),length(sidx))
μMat= zeros(length(sidx),1)
λ1idx=zeros(length(sidx)) 
λ2idx=zeros(length(sidx)) 
μidx=[]
λ1=λ  #Define Arrival Rate for Zone1
λ2=λ1  #Define Arrival Rate for Zone2                
μSame=μS
μDiff=μD
for s in 1:length(sidx)
    Zoneflag = sparti[s]
    Call1 = spartj[s]
    Call2 = spartk[s]
    RegionFrom= spartl[s]
    nextZoneflag = 0
    nextCall1=0
    nextCall2=0
    nexts=0
    RegionService= 0    
    nextRegionFrom=0

#  Arrival at region 1
    if Call1<max1demand
        nextCall1= Call1+1
        nextCall2= Call2
        if Zoneflag <=2
            nextZoneflag = 3 #don't use double equal
            nextRegionFrom=Zoneflag
        else 
            nextZoneflag = Zoneflag  #don't use double equal
            nextRegionFrom=RegionFrom
        end

        idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextRegionFrom)
        nexts= sidx[idx] 
        matrix[nexts,s].=matrix[nexts,s] .+ λ
        matrix[s,s]= matrix[s,s] - λ
         λ1idx[s]=1
    end
    #Arrival at region 2
    if Call2<max2demand
        nextCall1= Call1
        nextCall2= Call2+1       
      if Zoneflag <=2
            nextZoneflag = 4 #don't use double equal
            nextRegionFrom=Zoneflag
        else 
            nextZoneflag = Zoneflag  #don't use double equal
            nextRegionFrom=RegionFrom
        end
        idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .& (spartl.==nextRegionFrom)
        nexts= sidx[idx] 
        matrix[nexts,s].=matrix[nexts,s] .+ λ2
        matrix[s,s]= matrix[s,s] - λ2
        λ2idx[s]=1
    end
    
         #Service from Region to Region
    if Zoneflag == 3
        if Call1-1 >=2
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=3
            nextCall1=Call1-1
            nextCall2=Call2
            nextRegionFrom=1
            μidx=push!(μidx,1)        
        elseif Call2 == 1 && Call1-1 ==1
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag=1
            nextCall1=Call1-1
            nextCall2=Call2
            nextRegionFrom=1
            μidx=push!(μidx,1)            
        else
            RegionService=μMatr[1,RegionFrom]
            nextZoneflag= 4
            nextCall1= Call1-1
            nextCall2= Call2
            nextRegionFrom=1
            μidx=push!(μidx,2)
        end    
    elseif Zoneflag == 4
        if Call2-1 >=2
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=4
            nextCall1=Call1
            nextCall2=Call2-1
            nextRegionFrom=2
            μidx=push!(μidx,1)           
        elseif Call2-1 ==1 && Call1 ==1
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag=2
            nextCall1=Call1
            nextCall2=Call2-1
            nextRegionFrom=2
            μidx=push!(μidx,1)
        else
            RegionService=μMatr[2,RegionFrom]
            nextZoneflag= 3
            nextCall1= Call1
            nextCall2= Call2-1  
            nextRegionFrom=2
            μidx=push!(μidx,2)
        end
    else 
        RegionService= 0
        nextZoneflag=Zoneflag
        nextCall1=Call1
        nextCall2=Call2
        nextRegionFrom=RegionFrom
        μidx=push!(μidx,0)
    end    
  idx= (sparti.==nextZoneflag) .& (spartj.==nextCall1) .& (spartk.==nextCall2) .&(spartl.==nextRegionFrom)
  nexts= sidx[idx] 
  matrix[nexts,s]=matrix[nexts,s].+RegionService
  matrix[s,s]= matrix[s,s].- RegionService   
    μMat[s,1]= μMat[s,1] .+ RegionService
 end
matrix
μMat

362×1 Matrix{Float64}:
 0.0
 0.0
 5.0
 4.0
 5.0
 4.0
 5.0
 4.0
 5.0
 4.0
 5.0
 4.0
 5.0
 ⋮
 4.0
 5.0
 4.0
 5.0
 4.0
 5.0
 4.0
 5.0
 4.0
 5.0
 4.0
 5.0

In [80]:
matrix[length(sidx),:].= 1 #prep Ahat for matrix inversion
inverse=inv(matrix) #invert Ahat matrix
X=inverse[:,length(sidx)] #extract π from matrix
X=round.(X,digits=10) #round to 10 digits
A=sparti[1:length(sidx)] 
B=spartj[1:length(sidx)]
C=spartk[1:length(sidx)]
D=spartl[1:length(sidx)]
M=hcat(sidx,A,B,C,D,X)     #create readable matrix for state, sparti,spartj,spartk, and pi

362×6 Matrix{Any}:
   1  1   1   1  1  0.125373
   2  2   1   1  2  0.125373
   3  3   2   1  1  0.047239
   4  3   2   1  2  0.0506528
   5  3   2   2  1  0.0226434
   6  3   2   2  2  0.011819
   7  3   2   3  1  0.0118681
   8  3   2   3  2  0.00275777
   9  3   2   4  1  0.00648654
  10  3   2   4  2  0.000643479
  11  3   2   5  1  0.00365535
  12  3   2   5  2  0.000150145
  13  3   2   6  1  0.0021132
   ⋮                ⋮
 351  4  10   5  1  5.317e-7
 352  4  10   5  2  0.000475067
 353  4  10   6  1  4.617e-7
 354  4  10   6  2  0.000305138
 355  4  10   7  1  3.508e-7
 356  4  10   7  2  0.000181102
 357  4  10   8  1  2.426e-7
 358  4  10   8  2  9.95869e-5
 359  4  10   9  1  1.569e-7
 360  4  10   9  2  5.01331e-5
 361  4  10  10  1  2.392e-7
 362  4  10  10  2  2.21508e-5

In [1]:
Throughput=(λ)*sum(M[λ1idx.==1,6],M[λ2idx.==1,6])

LoadError: UndefVarError: λ1idx not defined

In [2]:
Gain=μMat'*X

LoadError: UndefVarError: μMat not defined

In [83]:
L=0
for s in 1:length(sidx)
  L=L+(B[s]+C[s]-2)*X[s]
end
L      

2.6193209142000016

In [84]:
W=L/Throughput   

0.7484109171496018

In [85]:
#Percentage of time with zero calls
Midx0= (spartj .+ spartk).==2
sum(M[Midx0.==1,6])

0.2507464972

In [87]:
#Percentage of time with one calls
Midx1= (spartj .+ spartk).==3
sum(M[Midx0.==1,6])

0.195783683